In [215]:
#import statements
import base64
import datetime
from urllib.parse import urlencode
import requests
import json
import csv

In [216]:
#Spotify API Client Object (SAPICO) definition
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string (not bytes)
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" #<base64 encoded client_id:client_secret>
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
#Accessed Spotify Endpoint: albums (https://developer.spotify.com/documentation/web-api/reference/albums/)
    
    def get_album_data(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
#Accessed Spotify Endpoint: artists (https://developer.spotify.com/documentation/web-api/reference/artists/)
    
    def get_artist_data(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def get_artist_genres(self, _id):
        return self.get_artist_data(_id)['genres']
    
#Accessed Spotify Endpoint: tracks (https://developer.spotify.com/documentation/web-api/reference/tracks/)
    
    def get_track_data(self, _id):
        return self.get_resource(_id, resource_type='tracks')
    
    def get_track_artists(self, _id):
        return self.get_track_data(_id)['artists']
    
    #UPDATE with iteration for multiple artists
    def get_track_artists_id(self, _id):
        return self.get_track_artists(_id)[0]['id']
    
    #Spotify API reference link: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/
    
    def get_track_audio_features(self, _id):
        return self.get_resource(_id, resource_type='audio-features')
    
    def get_track_specific_audio_feature(self, _id, specific_audio_feature='tempo'):
        return self.get_track_audio_features(_id)[specific_audio_feature]
    
#Accessed Spotify Endpoint: search (https://developer.spotify.com/documentation/web-api/reference/search/search/)
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        #OR or NOT are two operators
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)



In [217]:
#instantiate Spotify API client object
client_id = '' # '' = your client_id
client_secret = '' # '' = your client_secret
spotify = SpotifyAPI(client_id, client_secret)

In [218]:
#Andy Williams artistID: 4sj6D0zlMOl25nprDJBiU9
spotify.get_artist_data("4sj6D0zlMOl25nprDJBiU9")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4sj6D0zlMOl25nprDJBiU9'},
 'followers': {'href': None, 'total': 326299},
 'genres': ['adult standards', 'easy listening', 'mellow gold'],
 'href': 'https://api.spotify.com/v1/artists/4sj6D0zlMOl25nprDJBiU9',
 'id': '4sj6D0zlMOl25nprDJBiU9',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/cd88f73a20d4760a44e0e4f2378eefa9b1a0910e',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/4f44f5575e79e472e2078f95ec9e78768658250d',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/395d1258d17a0bd2bbead87dba510041c52ed4fa',
   'width': 160}],
 'name': 'Andy Williams',
 'popularity': 80,
 'type': 'artist',
 'uri': 'spotify:artist:4sj6D0zlMOl25nprDJBiU9'}

In [219]:
spotify.get_artist_genres("4sj6D0zlMOl25nprDJBiU9")

['adult standards', 'easy listening', 'mellow gold']

In [220]:
#Sign of the Times trackID: 5Ohxk2dO5COHF1krpoPigN (artist is Harry Styles)
spotify.get_track_data("5Ohxk2dO5COHF1krpoPigN")

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
    'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
    'id': '6KImCVD70vtIoJWnq6nGn3',
    'name': 'Harry Styles',
    'type': 'artist',
    'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}],
  'available_markets': ['AD',
   'AE',
   'AL',
   'AR',
   'AT',
   'AU',
   'BA',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'BY',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HR',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'KZ',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MD',
   'ME',
   'MK',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',

In [221]:
spotify.get_track_artists("5Ohxk2dO5COHF1krpoPigN")

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
  'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
  'id': '6KImCVD70vtIoJWnq6nGn3',
  'name': 'Harry Styles',
  'type': 'artist',
  'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}]

In [222]:
spotify.get_track_artists_id("5Ohxk2dO5COHF1krpoPigN")

'6KImCVD70vtIoJWnq6nGn3'

In [223]:
spotify.get_track_audio_features("5Ohxk2dO5COHF1krpoPigN")

{'danceability': 0.516,
 'energy': 0.595,
 'key': 5,
 'loudness': -4.63,
 'mode': 1,
 'speechiness': 0.0313,
 'acousticness': 0.0275,
 'instrumentalness': 0,
 'liveness': 0.109,
 'valence': 0.222,
 'tempo': 119.972,
 'type': 'audio_features',
 'id': '5Ohxk2dO5COHF1krpoPigN',
 'uri': 'spotify:track:5Ohxk2dO5COHF1krpoPigN',
 'track_href': 'https://api.spotify.com/v1/tracks/5Ohxk2dO5COHF1krpoPigN',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5Ohxk2dO5COHF1krpoPigN',
 'duration_ms': 340707,
 'time_signature': 4}

In [224]:
spotify.get_track_specific_audio_feature("5Ohxk2dO5COHF1krpoPigN", specific_audio_feature='valence')

0.222

In [225]:
#multiple artists test case
#1RUubW9fHtIYwjl588PrhZ - https://open.spotify.com/album/0lrmy4pJINsFzycJvttX2W?highlight=spotify:track:1RUubW9fHtIYwjl588PrhZ

spotify.get_track_artists("1RUubW9fHtIYwjl588PrhZ")

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2RdwBSPQiwcmiDo9kixcl8'},
  'href': 'https://api.spotify.com/v1/artists/2RdwBSPQiwcmiDo9kixcl8',
  'id': '2RdwBSPQiwcmiDo9kixcl8',
  'name': 'Pharrell Williams',
  'type': 'artist',
  'uri': 'spotify:artist:2RdwBSPQiwcmiDo9kixcl8'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/31TPClRtHm23RisEBtV3X7'},
  'href': 'https://api.spotify.com/v1/artists/31TPClRtHm23RisEBtV3X7',
  'id': '31TPClRtHm23RisEBtV3X7',
  'name': 'Justin Timberlake',
  'type': 'artist',
  'uri': 'spotify:artist:31TPClRtHm23RisEBtV3X7'}]

In [226]:
#TRACK INFO FOR API FETCHING#

##NON_CSV STUFF##

###COLUMNS FOR SHOWING info in FRONT END###
#explicit
#popularity

##CSV STUFF##

###COLUMNS FOR AUDIO FEATURES IN TRAINING DATA CSV### 
##(accessible via method: get_track_specific_audio_feature(self, _id, specific_audio_feature='tempo'))
#valence
#acousticness
#danceability
#duration_ms
#energy
#instrumentalness
#key
#liveliness
#loudness
#mode
#speechiness
#tempo

###other info in TRAINING DATA CSV###
#id ((###PRIMARY###))
#name
#year
#explicit
#popularity
#release_date
